# General overview

What agreements we have?

In [1]:
import os

base_path = 'OCR_output'

dirs = [
    d for d in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, d))
]

print(dirs)
print("Total directories:", len(dirs))


['Missouri', 'Alaska', 'New Hampshire', 'Illinois', 'Virginia', 'Washington', 'Vermont', 'South Dakota', 'Georgia', 'Connecticut', 'Alabama', 'Oklahoma', 'Colorado', 'Ohio', 'Maine', 'Mississippi', 'Utah', 'South Carolina', 'Delaware', 'Indiana', 'Arkansas', 'Iowa', 'New Jersey', 'Wisconsin', 'Michigan', 'Arizona', 'Massachusetts', 'Montana', 'New York', 'Wyoming', 'Minnesota', 'West Virginia', 'North Dakota', 'Florida', 'Kentucky', 'Louisiana', 'Maryland', 'Texas', 'Hawaii', 'Rhode Island', 'Kansas', 'Pennsylvania', 'North Carolina', 'Nebraska', 'Tennessee', 'Idaho', 'Oregon', 'Nevada', 'California', 'New Mexico']
Total directories: 50


We created 50 directories - for every state that is in USA - and then sort for these directories agreements. As we see we have 50 states cosidered. But does we have agreeemnets for each state?

In [2]:
state_counts = {}

for state in os.listdir(base_path):
    state_dir = os.path.join(base_path, state)

    if not os.path.isdir(state_dir):
        continue

    txt_count = sum(
        1 for f in os.listdir(state_dir)
        if f.lower().endswith(".txt")
    )

    state_counts[state] = txt_count
print("States directory:", state_counts, "\nAll states processed:", len(state_counts), 
      "\nNumber of states with agreements:", sum(1 for v in state_counts.values() if v > 0),
      "\nTotal agreements:", sum(state_counts.values()))


States directory: {'Missouri': 0, 'Alaska': 12, 'New Hampshire': 1, 'Illinois': 1, 'Virginia': 1, 'Washington': 4, 'Vermont': 11, 'South Dakota': 0, 'Georgia': 0, 'Connecticut': 2, 'Alabama': 10, 'Oklahoma': 0, 'Colorado': 0, 'Ohio': 5, 'Maine': 1, 'Mississippi': 0, 'Utah': 22, 'South Carolina': 0, 'Delaware': 0, 'Indiana': 3, 'Arkansas': 10, 'Iowa': 0, 'New Jersey': 1, 'Wisconsin': 2, 'Michigan': 4, 'Arizona': 17, 'Massachusetts': 1, 'Montana': 0, 'New York': 3, 'Wyoming': 0, 'Minnesota': 2, 'West Virginia': 0, 'North Dakota': 0, 'Florida': 0, 'Kentucky': 0, 'Louisiana': 0, 'Maryland': 4, 'Texas': 54, 'Hawaii': 4, 'Rhode Island': 1, 'Kansas': 0, 'Pennsylvania': 2, 'North Carolina': 2, 'Nebraska': 0, 'Tennessee': 0, 'Idaho': 1, 'Oregon': 1, 'Nevada': 0, 'California': 116, 'New Mexico': 0} 
All states processed: 50 
Number of states with agreements: 29 
Total agreements: 298


As we can see we have agreements for 29 states from 50 in total. We can say that lack of 21 is many but.. we also have 298 agreements at all so guge numner - enough to detailed analysis and further project.

**How is this data distributed on the map of the USA?**

Knowleadge Note:

The biggest three states in USA are: Alaska (AK), Texas (TX), California (CA).

In [3]:
import pandas as pd
import plotly.express as px

state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
}

df = pd.DataFrame([
    {
        "state": state_to_abbrev[state],
        "agreements": count
    }
    for state, count in state_counts.items()
    if state in state_to_abbrev
])

df["has_data"] = df["agreements"] > 0

fig = px.choropleth(
    df[df["agreements"] > 0],
    locations="state",
    locationmode="USA-states",
    color="agreements",
    scope="usa",
    color_continuous_scale=["lightblue", "darkblue"],
    title="States mentioned in Agreements Dataset",
    subtitle=f"Number of state in USA: 50, \nNumber of states with agreements in our task: {sum(1 for v in state_counts.values() if v > 0)}",
)
fig.add_choropleth(
    locations=df[df["agreements"] == 0]["state"],
    locationmode="USA-states",
    z=[0] * len(df[df["agreements"] == 0]),
    colorscale=[[0, "lightgrey"], [1, "lightgrey"]],
    showscale=False,
    name="No agreements"
)



fig.show()


In [4]:
df_sorted = df.sort_values("agreements", ascending=False)

fig = px.bar(
    df_sorted[df_sorted["agreements"] > 0],
    x="state",
    y="agreements",
    text="agreements",  # display number on top of bar
    title="Number of Agreements per State"
)

# show numbers on top of bars
fig.update_traces(textposition='outside')

# optional: tidy layout
fig.update_layout(
    xaxis_title="State",
    yaxis_title="Number of Agreements",
    coloraxis_showscale=False
)

fig.show()


As we can see we have agreements for the biggest stetes in USA (so we can say the most impotant agreements as it cover the largest area in case of surface). Moreover for **California and Texas  we have the largest amount of agreements**, Alaska has less agreements (we can susspect that this is because Alaska is located further from thers, it lays separetly in the context of whole USA map).

In top 3 there is also **Utah** - we can susppect that this is because it is the driest state and also have many national parks.

# Basic statisctics for each agreement

In [5]:
import json
import pandas as pd

data_entries = []

for state_folder in os.listdir(base_path):
  state_path = os.path.join(base_path, state_folder)
  for filename in os.listdir(state_path):
    if filename.endswith('.json'):
        file_path = os.path.join(state_path, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            json_output = json.load(f)

        num_pages = len(json_output['pages'])
        total_words = 0

        for page in json_output['pages']:
            for block in page['blocks']:
                for line in block['lines']:
                    total_words += len(line['words'])

        data_entries.append({
            'State': state_folder,
            'Filename': filename,
            'Num_Pages': num_pages,
            'Total_Words': total_words
        })

df = pd.DataFrame(data_entries)

**The most and the least crowded agreements**

In [6]:
df.sort_values(by="Num_Pages")

,State,Filename,Num_Pages,Total_Words
297,California,111102021.json,1,535
168,Texas,Texas_49.json,1,423
166,Texas,Texas_53.json,1,525
40,Alabama,Alabama_7.json,1,327
42,Ohio,Ohio_8.json,1,231
...,...,...,...,...
78,Arkansas,Arkansas_9.json,16,3932
132,Texas,Texas_8.json,19,1190
10,Alaska,Alaska_12.json,21,7368
240,California,California_16.json,21,5837


In [7]:
df.sort_values(by="Total_Words")

,State,Filename,Num_Pages,Total_Words
171,Hawaii,Hawaii_35.json,1,94
174,Hawaii,Hawaii_10.json,1,116
113,Maryland,Maryland_70.json,1,155
253,California,1September12.json,1,181
23,Vermont,Vermont_2.json,1,186
...,...,...,...,...
125,Texas,Texas_10.json,15,4307
164,Texas,Texas_12.json,14,4416
28,Vermont,Vermont_6.json,21,5837
240,California,California_16.json,21,5837


The range of pages for agreements is from 1 to 21. Range of words: 94-7368.

Mainly the more words the agreements have the more pages it have (what is obvious why).

Distribution of pages

In [8]:
page_counts = df.groupby('Num_Pages').size().reset_index(name='Num_Agreements')
import plotly.express as px

fig = px.bar(
    page_counts,
    x='Num_Pages',
    y='Num_Agreements',
    text='Num_Agreements',
    title='Number of Agreements by Number of Pages'
)

fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Number of Pages',
    yaxis_title='Number of Agreements'
)
fig.update_yaxes(range=[0, 70])
fig.show()



Although agreements are political files and we can think that they are really long based on this plot we see that agreements that we consider are mainly short. Most agreements have uo to 5 pages!

In [9]:
import pandas as pd
import plotly.express as px

# Count total agreements per state
state_order = df.groupby("State").size().sort_values(ascending=False).index

# Boxplot with x-axis sorted
fig = px.box(
    df,
    x="State",
    y="Num_Pages",
    points="all",
    category_orders={"State": state_order},  # sort x-axis
    title="Distribution of Number of Pages per State (sorted by total agreements)"
)

fig.update_layout(
    xaxis_title="State",
    yaxis_title="Number of Pages"
)

fig.show()




For states that have the largest amount og agreements we have the largest range of page number per agreement. Moreover states like Michigen and Pennsylvania have quite long agreemens. States on the right hand side from New Jersey have 1 page so they are not representative.

# Detailed Analysis for Each Agreement

What are the mostr common words among agreements? 

In [10]:
from collections import Counter
import os
import re
import pandas as pd

data_entries = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    for filename in os.listdir(state_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(state_path, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # split text into words (keep everything, including stop words)
            words = re.findall(r'\b\w+\b', text.lower())

            # get most common word
            if words:
                most_common_word, count = Counter(words).most_common(1)[0]
            else:
                most_common_word, count = None, 0

            data_entries.append({
                'State': state_folder,
                'Filename': filename,
                'Most_Common_Word': most_common_word,
                'Most_Common_Word_Count': count
            })

df_words_not_ommited = pd.DataFrame(data_entries)


In [11]:
df_words_not_ommited["Most_Common_Word"].unique()

array(['the', 'of', 'and', 'i', 'police', 'de'], dtype=object)

This words dont shock us as this is the most common words that aperes in senetences in the world not only in agreements. So to do better analysis we should ommit some most common words that appear in English languages.

In [12]:
stop_words = set([
    "and", "with", "it", "is", "are", "the", "a", "an", "of", "in", "on",
    "for", "to", "by", "at", "from", "as", "that", "this", "be", "or", "which",
    "both", "their", "from", "your", 'i', 'will', 'e', 's', 'de', 'r', 'à'
])


In [13]:
from collections import Counter
import os
import re

data_entries = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    for filename in os.listdir(state_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(state_path, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # split text into words, remove punctuation
            words = re.findall(r'\b\w+\b', text.lower())
            words_filtered = [w for w in words if w not in stop_words and w.isalpha()]

            # get most common word
            if words_filtered:
                most_common_word, count = Counter(words_filtered).most_common(1)[0]
            else:
                most_common_word, count = None, 0

            data_entries.append({
                'State': state_folder,
                'Filename': filename,
                'Most_Common_Word': most_common_word,
                'Most_Common_Word_Count': count
            })

df_words = pd.DataFrame(data_entries)



In [14]:
df_words["Most_Common_Word"].unique()

array(['oil', 'health', 'signatory', 'alaska', 'september', 'emergency',
       'alberta', 'cooperation', 'member', 'airport', 'british', 'state',
       'great', 'n', 'parties', 'energy', 'forest', 'mou', 'zev',
       'jurisdiction', 'information', 'relationship', 'police',
       'organization', 'lake', 'vermont', 'name', 'technology', 'states',
       'provinces', 'alabama', 'host', 'ofthe', 'education', 'board',
       'académie', 'utah', 'sichuan', 'understanding', 'economic',
       'gyeonggi', 'jiangxi', 'ministry', 'arkansas', 'license', 'taiwan',
       'supervisors', 'shall', 'michigan', 'sonora', 'arizona', 'party',
       'film', 'united', 'y', 'agreement', 'mexican', 'te', 'd', 'texas',
       'trade', 'manitoba', 'shaanxi', 'between', 'estado', 'driver',
       'fujian', 'authority', 'participants', 'exchange', 'memorandum',
       'we', 'development', 'line', 'finland', 'japan', 'affiliation',
       'provinceofcebu', 'nf', 'north', 'carbon', 'california', 'sweden',
   

The most common words covers topics like health area, economy (exchange, oil, water, tourism...), climate, countreis, materials, politics.

**Top words accross all documents**

In [64]:
stop_words = set([
    "and", "with", "it", "is", "are", "the", "a", "an", "of", "in", "on",
    "for", "to", "by", "at", "from", "as", "that", "this", "be", "or", "which",
    "both", "their", "from", "your", 'i', 'will', 'e', 's', 'de', 'r', 'à', 'y', 'shall',
    'missouri', 'alaska', 'new hampshire', 'illinois', 'virginia', 'washington', 'vermont', 'south dakota', 'georgia', 'connecticut', 'alabama', 
    'oklahoma', 'colorado', 'ohio', 'maine', 'mississippi', 'utah', 'south carolina', 'any',
    'delaware', 'indiana', 'arkansas', 'iowa', 'new jersey', 'wisconsin', 'michigan', 'arizona', 
    'massachusetts', 'montana', 'new york', 'wyoming', 'minnesota', 'west virginia', 'north dakota', 'florida', 'kentucky', 'louisiana', 'maryland', 'texas', 'hawaii', 'rhode island', 
    'kansas', 'pennsylvania', 'north carolina', 'nebraska', 'tennessee', 'idaho', 'oregon', 'nevada', 'california', 'new mexico'
])


In [65]:
from collections import Counter
import os
import re

data_entries = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    for filename in os.listdir(state_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(state_path, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # split text into words, remove punctuation
            words = re.findall(r'\b\w+\b', text.lower())
            words_filtered = [w for w in words if w not in stop_words and w.isalpha()]

            # get most common word
            if words_filtered:
                most_common_word, count = Counter(words_filtered).most_common(1)[0]
            else:
                most_common_word, count = None, 0

            data_entries.append({
                'State': state_folder,
                'Filename': filename,
                'Most_Common_Word': most_common_word,
                'Most_Common_Word_Count': count
            })

df_words = pd.DataFrame(data_entries)



Count of occurence of top 10 words across all agreements

In [66]:
df_words.groupby(["Most_Common_Word"])["Most_Common_Word_Count"] \
        .sum() \
        .sort_values(ascending=False) \
        .reset_index().head(10)


,Most_Common_Word,Most_Common_Word_Count
0,state,921
1,parties,423
2,information,422
3,energy,380
4,la,262
5,understanding,241
6,supervisors,233
7,participants,215
8,agreement,162
9,member,138


Files talk about states and parties, the most interest area across the files is energy.

**Top words per state**

In [67]:
from collections import Counter
import os
import re
import pandas as pd

state_word_totals = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    if not os.path.isdir(state_path):
        continue
    
    all_words = []
    
    for filename in os.listdir(state_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(state_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
            
            words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
            words_filtered = [w for w in words if w not in stop_words and len(w) > 2]
            all_words.extend(words_filtered)
    
    # count words per state
    counts = Counter(all_words)
    top3 = counts.most_common(3)
    
    for rank, (word, count) in enumerate(top3):
        state_word_totals.append({
            "State": state_folder,
            "Word": word,
            "Count": count,
            "rank": rank
        })

df_top3 = pd.DataFrame(state_word_totals)

In [68]:
df_top3["rank"] = df_top3.groupby("State").cumcount()


In [69]:
# total count per state
state_order = (
    df_top3.groupby("State")["Count"].sum()
    .sort_values(ascending=False)
    .index
)

# reorder df_top3 by this order
df_top3["State"] = pd.Categorical(df_top3["State"], categories=state_order, ordered=True)
df_top3 = df_top3.sort_values("State")


In [70]:
batch_size = 10
batches = [df_top3[i:i + batch_size*3] for i in range(0, len(state_order)*3, batch_size*3)]


In [71]:
import plotly.graph_objects as go

for i, batch in enumerate(batches):
    fig = go.Figure()
    
    for r in sorted(batch["rank"].unique()):
        df_r = batch[batch["rank"] == r]

        fig.add_bar(
            x=df_r["State"],
            y=df_r["Count"],
            text=df_r["Word"],
            textposition="outside",
            offsetgroup=str(r),
            showlegend=False
        )

    fig.update_layout(
        title=f"Top 3 words per state (batch {i+1})",
        xaxis_title="State",
        yaxis_title="Word occurrences",
        barmode="group",
        xaxis_tickangle=-45,
        height=600
    )

    fig.show()


As we see states talk abut agreements with other states, consider the parties (communication/cooperation we can guess), interest in economy - forest, enery, trade,carbon. The ionterest top word is for New Jersy - chihuahua.

**Detecting bigrams**

In [72]:
bigram_stop_phrases = set([
    "does not",
    "is not",
    "will not",
    "can not",
    "but not",
    "any other",
])


In [73]:
def tokenize_text(text):
    # Lowercase, split na słowa, ignoruj interpunkcję
    words = re.findall(r'\b\w+\b', text.lower())
    # Pomijaj stopwords i nie-alfabetyczne tokeny
    words_filtered = [w for w in words if w.isalpha() and w not in stop_words]
    return words_filtered


In [74]:
data_entries = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    if not os.path.isdir(state_path):
        continue

    for filename in os.listdir(state_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(state_path, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            words_filtered = tokenize_text(text)

            # Generate bigrams
            bigrams = [f"{words_filtered[i]} {words_filtered[i+1]}"
                       for i in range(len(words_filtered)-1)]

            # Filter undesired bigram phrases
            bigrams = [b for b in bigrams if b not in bigram_stop_phrases]

            # Count bigrams in this document
            bigram_counts = Counter(bigrams)

            data_entries.append({
                'State': state_folder,
                'Filename': filename,
                'Bigram_Counts': bigram_counts
            })

# ---------------------------
# Convert to DataFrame
# ---------------------------

df_bigrams = pd.DataFrame(data_entries)

# ---------------------------
# Global top bigrams
# ---------------------------

all_bigrams = Counter()
for counts in df_bigrams['Bigram_Counts']:
    all_bigrams.update(counts)

top_bigrams = all_bigrams.most_common(20)
print("Top 20 bigrams across all agreements:")
for bigram, count in top_bigrams:
    print(f"{bigram}: {count}")

Top 20 bigrams across all agreements:
memorandum understanding: 904
united states: 589
states america: 337
climate change: 326
economic development: 240
low carbon: 233
confidential information: 214
british columbia: 200
state united: 188
forest fire: 180
cooperation between: 173
each party: 163
greenhouse gas: 156
clean energy: 144
not limited: 142
great lakes: 138
zero emission: 132
other party: 131
government state: 130
written notice: 129


BIgrams say more - we see that interest area is enviroment, climate change and also cooperation between parties and states.

Examine without usa phreases:

In [75]:
bigram_stop_phrases = set([
    "does not",
    "is not",
    "will not",
    "can not",
    "but not",
    "any other",
    "states america",
    "united states"
    
])

In [76]:
data_entries = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    if not os.path.isdir(state_path):
        continue

    for filename in os.listdir(state_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(state_path, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            words_filtered = tokenize_text(text)

            # Generate bigrams
            bigrams = [f"{words_filtered[i]} {words_filtered[i+1]}"
                       for i in range(len(words_filtered)-1)]

            # Filter undesired bigram phrases
            bigrams = [b for b in bigrams if b not in bigram_stop_phrases]

            # Count bigrams in this document
            bigram_counts = Counter(bigrams)

            data_entries.append({
                'State': state_folder,
                'Filename': filename,
                'Bigram_Counts': bigram_counts
            })

# ---------------------------
# Convert to DataFrame
# ---------------------------

df_bigrams = pd.DataFrame(data_entries)

# ---------------------------
# Global top bigrams
# ---------------------------

all_bigrams = Counter()
for counts in df_bigrams['Bigram_Counts']:
    all_bigrams.update(counts)

top_bigrams = all_bigrams.most_common(20)
print("Top 20 bigrams across all agreements:")
for bigram, count in top_bigrams:
    print(f"{bigram}: {count}")

Top 20 bigrams across all agreements:
memorandum understanding: 904
climate change: 326
economic development: 240
low carbon: 233
confidential information: 214
british columbia: 200
state united: 188
forest fire: 180
cooperation between: 173
each party: 163
greenhouse gas: 156
clean energy: 144
not limited: 142
great lakes: 138
zero emission: 132
other party: 131
government state: 130
written notice: 129
best practices: 128
republic china: 124


In [77]:
state_bigram_totals = []

for state_folder in os.listdir(base_path):
    state_path = os.path.join(base_path, state_folder)
    if not os.path.isdir(state_path):
        continue
    
    all_words = []

    for filename in os.listdir(state_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(state_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()

            # tokenize
            words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
            words_filtered = [w for w in words if w not in stop_words and len(w) > 2]
            all_words.extend(words_filtered)
    
    # make bigrams
    bigrams = [f"{all_words[i]} {all_words[i+1]}" for i in range(len(all_words)-1)]
    # remove undesired bigram phrases
    bigrams = [b for b in bigrams if b not in bigram_stop_phrases]

    # count bigrams per state
    counts = Counter(bigrams)
    top3 = counts.most_common(3)  # exactly top 3
    
    for rank, (bigram, count) in enumerate(top3):
        state_bigram_totals.append({
            "State": state_folder,
            "Bigram": bigram,
            "Count": count,
            "rank": rank
        })

df_top3_bigrams = pd.DataFrame(state_bigram_totals)

In [78]:
batch_size = 10
batches = [df_top3_bigrams.iloc[i:i+batch_size] for i in range(0, len(df_top3_bigrams), batch_size)]


In [79]:
import plotly.graph_objects as go

# Assume df_top3_bigrams is already prepared
batch_size = 8
states = df_top3_bigrams["State"].unique()
batches = [df_top3_bigrams[df_top3_bigrams["State"].isin(states[i:i+batch_size])]
           for i in range(0, len(states), batch_size)]

for i, batch in enumerate(batches):
    fig = go.Figure()

    for r in sorted(batch["rank"].unique()):
        df_r = batch[batch["rank"] == r]

        fig.add_bar(
            x=df_r["State"],
            y=df_r["Count"],
            text=df_r["Bigram"],          # show bigram on bar
            textposition="outside",
            textfont=dict(size=14, color="black"),
            textangle=-80,                 # rotate text 80 degrees
            offsetgroup=str(r),
            name=f"Rank {r+1}"
        )

    fig.update_layout(
        title=f"Top 3 Bigrams per State (batch {i+1})",
        xaxis_title="State",
        yaxis_title="Occurrences",
        barmode="group",
        xaxis_tickangle=-45,             # rotate x-axis labels
        height=600
    )

    fig.show()


Bigrams give us the hint what the chihuahua can means for New Jersey:

**“Salud Chihuahua”** refers to the Secretaría de Salud Chihuahua, the Ministry of Health of the state of Chihuahua in Mexico. Many residents of Mexico (including those from the state of Chihuahua) live or work in the United States, including in New Jersey. In practice, this may involve:
cooperation between Mexican consulates in New Jersey and local health institutions,
preventive programs for migrants (such as vaccinations and occupational health),
but this does not constitute an institution-to-institution relationship between Salud Chihuahua and the authorities of New Jersey.
At times, organizations operating in the United States (including in New Jersey) also carry out health projects in Mexico, including in Chihuahua. In such cases, Salud Chihuahua may act as a local partner, but not as a partner of the state of New Jersey itself.